In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
def drop_columns_if_exist(df, columns_to_drop = ["latitude","longitude","elevation","status","STATE", "ID", "entry_id","UNIT", "CO", "field7","field8"]):
    # Check if each column exists in the DataFrame before dropping it
    columns_to_drop_existing = [col for col in columns_to_drop if col in df.columns]

    # Drop the existing columns from the DataFrame
    df.drop(columns=columns_to_drop_existing, inplace=True)

    return df

In [3]:
def group_and_average_by_minute(df, datetime_column_name):
    df = drop_columns_if_exist(df)
    # Convert the datetime column to datetime format
    df[datetime_column_name] = pd.to_datetime(df[datetime_column_name])

    # Set the datetime column as the DataFrame index for resampling
    df.set_index(datetime_column_name, inplace=True)

    # Resample the DataFrame to group by minute and calculate the average
    df_avg_per_minute = df.resample('1T').mean()

    # Reset the index to convert the datetime column back to a regular column
    df_avg_per_minute.reset_index(inplace=True)

    # Round the datetime column to the nearest minute
    df_avg_per_minute[datetime_column_name] = df_avg_per_minute[datetime_column_name].dt.round('1min')

    return df_avg_per_minute.dropna().reset_index(drop=True)


In [4]:
import pandas as pd

def merge_three_dataframes_by_datetime(df1, df2, df3, datetime_col_df1, datetime_col_df2, datetime_col_df3, timezone=None):
    # Convert the datetime columns to datetime format with the specified timezone (if provided)
    df1["created_at"] = pd.to_datetime(df1['created_at']) + pd.Timedelta(hours=1) # Change time from UTC to WAT
    df1[datetime_col_df1] = pd.to_datetime(df1[datetime_col_df1]).dt.tz_localize(None)
    df2[datetime_col_df2] = pd.to_datetime(df2[datetime_col_df2]).dt.tz_localize(None)
    df3[datetime_col_df3] = pd.to_datetime(df3[datetime_col_df3]).dt.tz_localize(None)

    # Perform the first inner merge between df1 and df2 based on the datetime columns
    merged_df1 = pd.merge(df1, df2, left_on=datetime_col_df1, right_on=datetime_col_df2, how='inner')

    # Drop the redundant datetime column from df2 as it's no longer needed in the first merged DataFrame
    merged_df1.drop(columns=[datetime_col_df2], inplace=True)

    # Perform the second inner merge between the first merged DataFrame and df3 based on their datetime columns
    merged_df = pd.merge(merged_df1, df3, left_on=datetime_col_df1, right_on=datetime_col_df3, how='inner')

    # Drop the redundant datetime column from df3 as it's no longer needed in the final merged DataFrame
    merged_df.drop(columns=[datetime_col_df3], inplace=True)

    return merged_df


In [5]:
alpha_dataset = pd.DataFrame()
data_length_calib_device = []
data_length_low_cost_sensors = []

for i in range(6):
    print("Processing ################# Day {} ############################".format(i+1))
    alpha = pd.read_csv("Day 0{}/Alpha.csv".format(i+1))
    co2 =  pd.read_csv("Day 0{}/CO2.csv".format(i+1))
    co =  pd.read_csv("Day 0{}/CO.csv".format(i+1))
    data_length_calib_device.append(co.shape[0])
    data_length_low_cost_sensors.append(alpha.shape[0])
    
    
    co2 = co2.rename(columns = {"SHOW":"CO2"})
    co = co.rename(columns = {"SHOW":"CO"})
    co.CO  = co.CO.fillna(0)

    alpha = group_and_average_by_minute(alpha, 'created_at')
    co2 = group_and_average_by_minute(co2, 'TIME')
    co = group_and_average_by_minute(co, 'TIME')

    merged_data = merge_three_dataframes_by_datetime(alpha, co2, co, 'created_at', 'TIME', 'TIME')   
    merged_data = merged_data[30:].reset_index(drop=True)
    alpha_dataset = alpha_dataset.append(merged_data)
    
for i in range(6,9):
    print("Processing ################# Day {} ############################".format(i+1))
    alpha = pd.read_csv("Day 0{}/Alpha.csv".format(i+1))
    co2 =  pd.read_csv("Day 0{}/CO2/CO2.csv".format(i+1))
    co =  pd.read_csv("Day 0{}/CO/CO.csv".format(i+1))
    data_length_calib_device.append(co.shape[0])
    data_length_low_cost_sensors.append(alpha.shape[0])

    co2 = co2.rename(columns = {"SHOW":"CO2"})
    co = co.rename(columns = {"SHOW":"CO"})
    co.CO  = co.CO.fillna(0)

    alpha = group_and_average_by_minute(alpha, 'created_at')
    co2 = group_and_average_by_minute(co2, 'TIME')
    co = group_and_average_by_minute(co, 'TIME')

    merged_data = merge_three_dataframes_by_datetime(alpha, co2, co, 'created_at', 'TIME', 'TIME')    
    merged_data = merged_data[30:].reset_index(drop=True)
    alpha_dataset = alpha_dataset.append(merged_data)
    

alpha_dataset = alpha_dataset.reset_index(drop=True)
alpha_dataset = alpha_dataset.rename(columns={'SHOW': 'CO2',"field1":"Temperature", "field2":"Humidity", "field3":"MQ7_analog", "field4":"MQ9_analog", "field5":"MG811_analog", "field6":"MQ135_analog", "field7":"MG811_Digital", "field8":"MQ7_Digital"})
alpha_dataset.to_csv("alpha.csv", index=False)
alpha_dataset.tail()

Processing ################# Day 1 ############################
Processing ################# Day 2 ############################
Processing ################# Day 3 ############################
Processing ################# Day 4 ############################
Processing ################# Day 5 ############################
Processing ################# Day 6 ############################
Processing ################# Day 7 ############################
Processing ################# Day 8 ############################
Processing ################# Day 9 ############################


,created_at,Temperature,Humidity,MQ7_analog,MQ9_analog,MG811_analog,MQ135_analog,CO2
2809,2023-06-29 14:43:00,30.640,67.470,2395.5,3520.0,1698.5,2071.5,583.583333
2810,2023-06-29 14:44:00,30.665,67.985,2399.5,3503.5,1697.5,2076.5,584.500000
2811,2023-06-29 14:45:00,30.760,70.285,2413.5,3541.5,1692.5,2084.0,583.416667
2812,2023-06-29 14:46:00,30.920,69.880,2409.5,3515.0,1692.0,2083.0,585.600000
2813,2023-06-29 15:40:00,31.620,70.530,2380.0,3460.0,1731.0,2009.0,573.166667


In [6]:
print("sensors", sum(data_length_low_cost_sensors)*4)
print("Calibration device",sum(data_length_calib_device))

print("Estimated Total data points gathered", sum(data_length_low_cost_sensors)*4 + sum(data_length_calib_device))

sensors 30336
Calibration device 12923
Estimated Total data points gathered 43259


In [7]:
Beta_dataset = pd.DataFrame()
for i in range(6):
    print("Processing ################# Day {} ############################".format(i+1))
    Beta = pd.read_csv("Day 0{}/Beta.csv".format(i+1))
    co2 =  pd.read_csv("Day 0{}/CO2.csv".format(i+1))
    co =  pd.read_csv("Day 0{}/CO.csv".format(i+1))
    
    co2 = co2.rename(columns = {"SHOW":"CO2"})
    co = co.rename(columns = {"SHOW":"CO"})
    co.CO  = co.CO.fillna(0)

    Beta = group_and_average_by_minute(Beta, 'created_at')
    co2 = group_and_average_by_minute(co2, 'TIME')
    co = group_and_average_by_minute(co, 'TIME')

    merged_data = merge_three_dataframes_by_datetime(Beta, co2, co, 'created_at', 'TIME', 'TIME')   
    merged_data = merged_data[30:].reset_index(drop=True)
    Beta_dataset = Beta_dataset.append(merged_data)
    
for i in range(6,9):
    print("Processing ################# Day {} ############################".format(i+1))
    Beta = pd.read_csv("Day 0{}/Beta.csv".format(i+1))
    co2 =  pd.read_csv("Day 0{}/CO2/CO2.csv".format(i+1))
    co =  pd.read_csv("Day 0{}/CO/CO.csv".format(i+1))

    co2 = co2.rename(columns = {"SHOW":"CO2"})
    co = co.rename(columns = {"SHOW":"CO"})
    co.CO  = co.CO.fillna(0)

    Beta = group_and_average_by_minute(Beta, 'created_at')
    co2 = group_and_average_by_minute(co2, 'TIME')
    co = group_and_average_by_minute(co, 'TIME')

    merged_data = merge_three_dataframes_by_datetime(Beta, co2, co, 'created_at', 'TIME', 'TIME')    
    merged_data = merged_data[30:].reset_index(drop=True)
    Beta_dataset = Beta_dataset.append(merged_data)
    

Beta_dataset = Beta_dataset.reset_index(drop=True)
Beta_dataset = Beta_dataset.rename(columns={'SHOW': 'CO2',"field1":"Temperature", "field2":"Humidity", "field3":"MQ7_analog", "field4":"MQ9_analog", "field5":"MG811_analog", "field6":"MQ135_analog", "field7":"MG811_Digital", "field8":"MQ7_Digital"})
Beta_dataset.to_csv("Beta.csv", index=False)
Beta_dataset.tail()

Processing ################# Day 1 ############################
Processing ################# Day 2 ############################
Processing ################# Day 3 ############################
Processing ################# Day 4 ############################
Processing ################# Day 5 ############################
Processing ################# Day 6 ############################
Processing ################# Day 7 ############################
Processing ################# Day 8 ############################
Processing ################# Day 9 ############################


,created_at,Temperature,Humidity,MQ7_analog,MQ9_analog,MG811_analog,MQ135_analog,CO2
2919,2023-06-29 14:43:00,29.315,68.575,3251.0,4808.0,3515.5,3567.0,583.583333
2920,2023-06-29 14:44:00,29.345,70.175,3232.5,4784.5,3499.0,3555.5,584.500000
2921,2023-06-29 14:45:00,29.390,72.345,3247.5,4787.0,3511.5,3560.5,583.416667
2922,2023-06-29 14:46:00,29.470,71.060,3215.0,4746.0,3497.0,3540.0,585.600000
2923,2023-06-29 15:40:00,30.290,69.660,3143.5,4711.0,3554.5,3587.0,573.166667


In [8]:
Charlie_dataset = pd.DataFrame()
for i in range(6):
    print("Processing ################# Day {} ############################".format(i+1))
    Charlie = pd.read_csv("Day 0{}/Charlie.csv".format(i+1))
    co2 =  pd.read_csv("Day 0{}/CO2.csv".format(i+1))
    co =  pd.read_csv("Day 0{}/CO.csv".format(i+1))
    
    co2 = co2.rename(columns = {"SHOW":"CO2"})
    co = co.rename(columns = {"SHOW":"CO"})
    co.CO  = co.CO.fillna(0)

    Charlie = group_and_average_by_minute(Charlie, 'created_at')
    co2 = group_and_average_by_minute(co2, 'TIME')
    co = group_and_average_by_minute(co, 'TIME')

    merged_data = merge_three_dataframes_by_datetime(Charlie, co2, co, 'created_at', 'TIME', 'TIME')   
    merged_data = merged_data[30:].reset_index(drop=True)
    Charlie_dataset = Charlie_dataset.append(merged_data)
    
for i in range(6,9):
    print("Processing ################# Day {} ############################".format(i+1))
    Charlie = pd.read_csv("Day 0{}/Charlie.csv".format(i+1))
    co2 =  pd.read_csv("Day 0{}/CO2/CO2.csv".format(i+1))
    co =  pd.read_csv("Day 0{}/CO/CO.csv".format(i+1))

    co2 = co2.rename(columns = {"SHOW":"CO2"})
    co = co.rename(columns = {"SHOW":"CO"})
    co.CO  = co.CO.fillna(0)

    Charlie = group_and_average_by_minute(Charlie, 'created_at')
    co2 = group_and_average_by_minute(co2, 'TIME')
    co = group_and_average_by_minute(co, 'TIME')

    merged_data = merge_three_dataframes_by_datetime(Charlie, co2, co, 'created_at', 'TIME', 'TIME')    
    merged_data = merged_data[30:].reset_index(drop=True)
    Charlie_dataset = Charlie_dataset.append(merged_data)
    

Charlie_dataset = Charlie_dataset.reset_index(drop=True)
Charlie_dataset = Charlie_dataset.rename(columns={'SHOW': 'CO2',"field1":"Temperature", "field2":"Humidity", "field3":"MQ7_analog", "field4":"MQ9_analog", "field5":"MG811_analog", "field6":"MQ135_analog", "field7":"MG811_Digital", "field8":"MQ7_Digital"})
Charlie_dataset.to_csv("Charlie.csv", index=False)
Charlie_dataset.tail()

Processing ################# Day 1 ############################
Processing ################# Day 2 ############################
Processing ################# Day 3 ############################
Processing ################# Day 4 ############################
Processing ################# Day 5 ############################
Processing ################# Day 6 ############################
Processing ################# Day 7 ############################
Processing ################# Day 8 ############################
Processing ################# Day 9 ############################


,created_at,Temperature,Humidity,MQ7_analog,MQ9_analog,MG811_analog,MQ135_analog,CO2
2856,2023-06-29 14:43:00,30.420000,66.320000,3374.000000,3243.000000,2931.000000,2481.0,583.583333
2857,2023-06-29 14:44:00,30.275000,68.545000,3281.500000,3210.500000,2885.500000,2439.5,584.500000
2858,2023-06-29 14:45:00,30.476667,69.843333,3386.333333,3257.333333,2913.333333,2474.0,583.416667
2859,2023-06-29 14:46:00,30.670000,69.190000,3337.000000,3232.000000,2905.000000,2461.0,585.600000
2860,2023-06-29 15:40:00,31.420000,67.975000,3395.500000,3299.500000,2882.000000,2461.5,573.166667


In [9]:
alpha_dataset.min()#.head()

created_at      2023-05-01 03:29:00
Temperature                  27.665
Humidity                       52.2
MQ7_analog                   2380.0
MQ9_analog                   1098.5
MG811_analog                 1353.0
MQ135_analog                 1186.5
CO2                      573.166667
dtype: object

In [10]:
Beta_dataset.min()#.head()

created_at      2023-05-01 03:30:00
Temperature                   23.44
Humidity                  59.903333
MQ7_analog                   3109.5
MQ9_analog                   3555.0
MG811_analog                 2412.5
MQ135_analog                 3156.5
CO2                      573.166667
dtype: object

In [11]:
Charlie_dataset.min()#.head()

created_at      2023-05-01 03:29:00
Temperature                   23.93
Humidity                     60.935
MQ7_analog                   3128.0
MQ9_analog                   3210.5
MG811_analog                 2479.0
MQ135_analog                 2381.0
CO2                      573.166667
dtype: object

In [12]:
alpha_dataset.describe()

,Temperature,Humidity,MQ7_analog,MQ9_analog,MG811_analog,MQ135_analog,CO2
count,2814.000000,2814.000000,2814.000000,2814.000000,2814.000000,2814.000000,2814.000000
mean,31.830263,62.862624,3630.742715,2203.750118,4279.351042,2548.040393,611.465640
std,1.233422,5.447757,1082.643577,985.415917,1337.378345,727.726906,16.391828
min,27.665000,52.200000,2380.000000,1098.500000,1353.000000,1186.500000,573.166667
25%,31.715000,58.905000,2863.625000,1498.541667,3171.125000,2072.625000,600.000000
50%,31.910000,61.455000,3341.000000,1729.250000,4425.166667,2682.000000,608.000000
75%,32.540000,66.875000,4023.875000,2810.875000,5184.000000,2935.500000,621.000000
max,33.850000,77.395000,9545.500000,7823.000000,7919.000000,5594.000000,677.000000


In [13]:
Beta_dataset.describe()

,Temperature,Humidity,MQ7_analog,MQ9_analog,MG811_analog,MQ135_analog,CO2
count,2924.000000,2924.000000,2924.000000,2924.000000,2924.000000,2924.000000,2924.000000
mean,28.244193,74.063788,3983.875456,4497.156065,3407.310078,4000.155894,611.855441
std,1.381560,5.494902,610.322973,1058.452935,643.839651,518.073859,16.673599
min,23.440000,59.903333,3109.500000,3555.000000,2412.500000,3156.500000,573.166667
25%,27.775000,71.173750,3455.375000,3905.500000,2794.250000,3589.000000,600.000000
50%,28.400000,74.185000,3867.750000,4085.500000,3246.250000,3819.250000,608.000000
75%,29.100000,75.310000,4424.625000,4832.000000,4067.000000,4369.500000,621.500000
max,32.305000,93.525000,8021.000000,10433.000000,5119.000000,6285.000000,677.000000


In [14]:
Charlie_dataset.describe()

,Temperature,Humidity,MQ7_analog,MQ9_analog,MG811_analog,MQ135_analog,CO2
count,2861.000000,2861.000000,2861.000000,2861.000000,2861.000000,2861.000000,2861.000000
mean,29.039685,71.574650,5054.418968,5163.769253,4292.091926,3738.785564,611.916367
std,1.301354,5.351279,951.077221,924.096242,949.952667,644.314595,16.798683
min,23.930000,60.935000,3128.000000,3210.500000,2479.000000,2381.000000,573.166667
25%,28.690000,68.715000,4316.000000,4358.000000,3196.000000,3199.500000,600.000000
50%,29.230000,71.250000,4982.000000,5366.500000,4471.500000,3778.000000,608.000000
75%,29.570000,73.975000,5826.000000,5693.500000,4981.500000,4214.500000,622.000000
max,32.720000,90.550000,8136.000000,8141.000000,6052.000000,5105.000000,677.000000
